In [1]:
from learning import learn_pool

'''rewards_config = {'play0' : 1, 'play1' : 3, 'play2' : 9, 'play3' : 27, 'play4' : 81,
                  'baseline' :3,
                  'discard_last_copy' : -100, 'discard_extra' : 0.5,
                  'hint_last_copy' : 0.2, 'hint_penalty' : 0.1,  'hint_playable' : 0.2,
                  'use_hamming' : True, 'loose_life' : -50}'''

##########################################################################
env_config = {'environment_name' : 'Hanabi-Small', 'num_players' : 2, 
              'use_custom_rewards' : False, 'open_hands' : False}
rewards_config = {}
model_config_base = {'scope' : 'agent_test', 'nenvs'  : 32, 
           'fc_input_layers' : [128, ], 'noisy_fc' : False, 'v_net' : 'copy',
           'gamma' : 0.99, 'ent_coef' : 0.0, 'vf_coef' : 1, 'cliprange' : 0.2,
           'max_grad_norm' : None, 'k' : 8,  
           'lr' : 1e-4, 'lr_half_period' : 50000, 'anneal_lr' : False,
           'normalize_advs': True, 'layer_norm' : False,
           }


method = 'self play'

In [2]:
import matplotlib.pyplot as plt
import os
%matplotlib inline
def plot_log(folder, runname,):
    agentscope = os.listdir(folder + runname)
    path  = folder + runname + '/' + agentscope + '/summary/'
    log = os.listdir(path)[0]
    for val in tf.train.summary_iterator(path + log):
        print(val)

# Baselines

#### NO EXP


In [3]:
folder = './experiments/baselines/noexploration/'
model_config_base = {'scope' : 'agent_test', 'nenvs'  : 32, 
           'fc_input_layers' : [128, ], 'noisy_fc' : False, 'v_net' : 'copy',
           'gamma' : 0.99, 'ent_coef' : 0.0, 'vf_coef' : 1, 'cliprange' : 0.2,
           'max_grad_norm' : None, 'k' : 8,  
           'lr' : 1e-4, 'lr_half_period' : 50000, 'anneal_lr' : False,
           'normalize_advs': True, 'layer_norm' : False,
           }

folder = './experiments/test/'

learn_pool.run_experiment('test', [model_config_base], env_config, rewards_config, 500, 
                          folder = folder, method = method, nsteps = 8,)
    




GAME PARAMETERS: 
  observation length = 171
  number of actions = 11
  number of players = 2




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use `tf.random.categorical` instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Could not load model "agent_test" at ./experiments/test/Hanabi-Small-2/test/agent_test/model/




Savinig models after 100 epochs of training
Average env speed is 2240 ts/second

Savinig models after 200 epochs of training
Average env speed is 2252 ts/second
---------------200---------------
Matrix of models performance with each others:
            agent_test
agent_test    0.896875
---------------------------------

Savinig models after 300 epochs of training
Average env speed is 2335 ts/second

Savinig models after 400 epochs of training
Average env speed is 2100 ts/second
---------------400---------------
Matrix of models performance with each others:
            agent_test
agent_test         1.0
---------------------------------


tensorflow.python.client.session.Session

In [8]:
tf.compat.v1.reset_default_graph

<function tensorflow.python.framework.ops.reset_default_graph()>

### shared copy

In [5]:

folder = './experiments/baselines/vnet/'
k = 24
steps = 8
epochs = 20000

for vnet in ['shared', 'copy']:
    mc = dict(model_config_base)
    mc['k'] = k
    mc['v_net'] = vnet
    runname = '%s_k:%d_st:%s' % (v_net, ent, k, steps)
    print(runname)
    print(mc)
        
    learn_pool.run_experiment(runname, [mc], env_config, rewards_config, epochs, 
                              folder = folder, method = method, nsteps = steps,)

NameError: name 'v_net' is not defined

#### ENTROPY

In [ ]:
folder = './experiments/baselines/exploration/'

k, steps, epochs, v_net = best

for ent in [0.01, 0.05]:
    mc = dict(model_config_base)
    mc['k'] = k
    mc['ent_coef'] = ent
    mc['noisy_fc'] = False
    runname = 'ent%.2f_k:%d_st:%s' % (ent, k, steps)
    print(runname)
    print(mc)
        
    #learn_pool.run_experiment(runname, [mc], env_config, rewards_config, epochs, 
    #                              folder = folder, method = method, nsteps = steps,)

#### NOISY


In [ ]:
folder = './experiments/baselines/exploration/'

k, steps, epochs, v_net = best

mc = dict(model_config_base)
mc['k'] = k
mc['ent_coef'] = 0
mc['noisy_fc'] = True
runname = 'noisy.2f_k:%d_st:%s' % (ent, k, steps)
print(runname)
print(mc)
        
#learn_pool.run_experiment(runname, [mc], env_config, rewards_config, epochs, 
#                              folder = folder, method = method, nsteps = steps,)    
    

### fc or lstm

In [ ]:
k, steps, epochs, v_net, ent, noisy = best

for fc, lstm in [([128], []),
                 ([128, 128], []),
                 ([128], [128])
                 ([128], [128, 128])]:
    
    
    mc = dict(model_config_base)
    mc['k'] = k
    mc['ent_coef'] = ent
    mc['noisy_fc'] = noisy
    
    runname = 'fc' + 'x'.join(fc) +  '_lstm' + 'x'.join(lstm) + 'k:' + str(k) + 'st:' % str(steps)
    if noisy:
        runname += '_noisy'
    else:
        runname += 'ent:' + str(ent)
    print(runname)
    print(mc)
        
    #learn_pool.run_experiment(runname, [mc], env_config, rewards_config, epochs, 
    #                              folder = folder, method = method, nsteps = steps,)   

#### noisy VS exploration for shared and copy

In [10]:
mc['k'] = 8
mc['v_net'] = 'shared'
runname = 'fc:128_%s_k:%d_noisy_st:36' % (mc['v_net'], mc['k'])
learn_pool.run_experiment(runname, [mc], env_config, rewards_config, 5000, 
                          folder = folder, method = method, nsteps = steps,)

GAME PARAMETERS: 
  observation length = 171
  number of actions = 11
  number of players = 2
Successfully loaded model "agent_test":
  "agent_test" was trained for 1900 epochs, using 4599744 timesteps
  "agent_test" has following parameters: k = 8, cliprange = 0.2, lr = 0.000100
  "agent_test" has following rewards_config: {}

Savinig models after 100 epochs of training
Average env speed is 3684 ts/second

Savinig models after 200 epochs of training
Average env speed is 3649 ts/second
---------------200---------------
Matrix of models performance with each others:
            agent_test
agent_test     1.76875
---------------------------------

Savinig models after 300 epochs of training
Average env speed is 3756 ts/second

Savinig models after 400 epochs of training
Average env speed is 3572 ts/second
---------------400---------------
Matrix of models performance with each others:
            agent_test
agent_test     1.75625
---------------------------------

Savinig models after 500

KeyboardInterrupt: 